# Vit for classification in CIFAR-10

In [1]:
%load_ext autoreload
%autoreload 2
!nvidia-smi
!which python

Thu Oct  3 14:39:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000004:04:00.0 Off |                    0 |
| N/A   42C    P0              45W / 184W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
try:
    from ViT.train import train
    from ViT.utils import cifar_train_set, cifar_val_set
    from ViT.model import *
except:
    from train import train
    from utils import cifar_train_set, cifar_val_set
    from model import *

from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F

import datetime

train_loader = DataLoader(cifar_train_set, 256, shuffle=True, drop_last=False, pin_memory=True)
val_loader = DataLoader(cifar_val_set, 500, shuffle=True, drop_last=False, pin_memory=True)

import os
if not os.path.exists("ViT/log"):
    os.makedirs("ViT/log")

def timestr():
    now = datetime.datetime.now()
    return now.strftime("%Y%m%d_%H%M%S")

def get_outdir(time_str):
    outdir = f"ViT/log/{time_str}.out"
    return outdir

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Here is the hyperparameters

epochs = 50
patch_size = 8
embed_dim = 256
n_layers = 6
heads = 8
attn_dim = 512
mlp_dim = None # default to 4*embed_dim
pool = 'cls'
dropout = 0.0

model = ViT(image_size=32, patch_size=patch_size, num_classes=10, embed_dim=embed_dim, n_layers=n_layers, heads=heads, attn_dim=attn_dim, mlp_dim=mlp_dim, pool=pool, dropout=dropout)

optimizer = torch.optim.Adam(model.parameters(), lr=2e-4, betas=(0.9, 0.999), weight_decay=5e-4)

time_str = timestr()

print(f"Time string: {time_str}")

# print the model and the number of parameters
# print(model.transformer)
print(f'The model has {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters')

train(epochs=epochs, model=model, optimizer=optimizer, criterion=nn.CrossEntropyLoss(), 
      train_loader=train_loader, val_loader=val_loader, outdir=get_outdir(time_str))\

# save model
torch.save(model.state_dict(), f"ViT/models/{time_str}.pth")

Time string: 20241002_154301
The model has 6,356,234 trainable parameters


Epoch 2/50:   0%|                                                                                | 0/196 [00:00<?, ?it/s]

Epoch 1/50, Train Loss: 2.0066, Train Accuracy: 24.60%, Val Loss: 1.8415, Val Accuracy: 31.70%


Epoch 3/50:   0%|                                                                                | 0/196 [00:00<?, ?it/s]

Epoch 2/50, Train Loss: 1.7390, Train Accuracy: 35.72%, Val Loss: 1.6409, Val Accuracy: 39.93%


...

Epoch 49/50:   1%|▎                                                    | 1/196 [00:00<00:32,  6.02it/s, Train Loss=0.988]

Epoch 48/50, Train Loss: 1.0321, Train Accuracy: 63.07%, Val Loss: 1.0732, Val Accuracy: 61.37%


Epoch 50/50:   1%|▎                                                     | 1/196 [00:00<00:32,  6.05it/s, Train Loss=1.09]

Epoch 49/50, Train Loss: 1.0256, Train Accuracy: 63.22%, Val Loss: 1.0755, Val Accuracy: 61.80%


Epoch 50/50, Train Loss: 1.0150, Train Accuracy: 63.70%, Val Loss: 1.1179, Val Accuracy: 60.34%


In [3]:
# Here is the hyperparameters

epochs = 50
patch_size = 8
embed_dim = 256
n_layers = 6
heads = 8
attn_dim = 512
mlp_dim = None # default to 4*embed_dim
pool = 'cls'
dropout = 0.0

model = ViT(image_size=32, patch_size=patch_size, num_classes=10, embed_dim=embed_dim, n_layers=n_layers, heads=heads, attn_dim=attn_dim, mlp_dim=mlp_dim, pool=pool, dropout=dropout)

# load model
model.load_state_dict(torch.load("ViT.pth"))

optimizer = torch.optim.Adam(model.parameters(), lr=2e-4, betas=(0.9, 0.999), weight_decay=5e-4)

time_str = timestr()

print(f"Time string: {time_str}")

# print the model and the number of parameters
# print(model.transformer)
print(f'The model has {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters')

train(epochs=epochs, model=model, optimizer=optimizer, criterion=nn.CrossEntropyLoss(), 
      train_loader=train_loader, val_loader=val_loader, outdir=get_outdir(time_str))

# save model
torch.save(model.state_dict(), f"ViT/models/{time_str}.pth")

print(f"models saved to ViT/models/{time_str}.pth")

Epoch 1/50:   0%|                                                                               | 0/196 [00:00<?, ?it/s]

Time string: 20241003_113452
The model has 6,356,234 trainable parameters


Epoch 2/50:   0%|                                                                               | 0/196 [00:00<?, ?it/s]

Epoch 1/50, Train Loss: 1.0165, Train Accuracy: 63.69%, Val Loss: 1.0695, Val Accuracy: 61.99%


Epoch 3/50:   1%|▎                                                    | 1/196 [00:00<00:37,  5.26it/s, Train Loss=0.983]

...

Epoch 49/50:   1%|▎                                                   | 1/196 [00:00<00:37,  5.26it/s, Train Loss=0.747]

Epoch 48/50, Train Loss: 0.8056, Train Accuracy: 71.18%, Val Loss: 0.9458, Val Accuracy: 66.86%


Epoch 50/50:   0%|                                                                              | 0/196 [00:00<?, ?it/s]

Epoch 49/50, Train Loss: 0.8081, Train Accuracy: 71.16%, Val Loss: 0.9222, Val Accuracy: 67.52%


Epoch 50/50, Train Loss: 0.8030, Train Accuracy: 71.42%, Val Loss: 0.9684, Val Accuracy: 66.04%


In [3]:
# Here is the hyperparameters

epochs = 50
patch_size = 8
embed_dim = 256
n_layers = 6
heads = 8
attn_dim = 512
mlp_dim = None # default to 4*embed_dim
pool = 'cls'
dropout = 0.0

model = ViT(image_size=32, patch_size=patch_size, num_classes=10, embed_dim=embed_dim, n_layers=n_layers, heads=heads, attn_dim=attn_dim, mlp_dim=mlp_dim, pool=pool, dropout=dropout)

# load model
model.load_state_dict(torch.load(f"ViT.pth"))

optimizer = torch.optim.Adam(model.parameters(), lr=2e-4, betas=(0.9, 0.999), weight_decay=5e-4)

time_str = timestr()

print(f"Time string: {time_str}")

# print the model and the number of parameters
# print(model.transformer)
print(f'The model has {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters')

train(epochs=epochs, model=model, optimizer=optimizer, criterion=nn.CrossEntropyLoss(), 
      train_loader=train_loader, val_loader=val_loader, outdir=get_outdir(time_str))

torch.save(model.state_dict(), f"ViT/models/{time_str}.pth")

print(f"models saved to ViT/models/{time_str}.pth")

Epoch 1/50:   0%|                                                                               | 0/196 [00:00<?, ?it/s]

Time string: 20241003_124314
The model has 6,356,234 trainable parameters


Epoch 2/50:   0%|                                                                               | 0/196 [00:00<?, ?it/s]

Epoch 1/50, Train Loss: 0.8029, Train Accuracy: 71.31%, Val Loss: 0.9669, Val Accuracy: 66.42%


Epoch 3/50:   0%|                                                                               | 0/196 [00:00<?, ?it/s]

...

Epoch 49/50:   1%|▎                                                   | 1/196 [00:00<00:37,  5.18it/s, Train Loss=0.625]

Epoch 48/50, Train Loss: 0.6432, Train Accuracy: 76.85%, Val Loss: 0.8781, Val Accuracy: 69.74%


Epoch 50/50:   1%|▎                                                    | 1/196 [00:00<00:37,  5.22it/s, Train Loss=0.73]

Epoch 49/50, Train Loss: 0.6323, Train Accuracy: 77.50%, Val Loss: 0.8879, Val Accuracy: 69.51%


Epoch 50/50, Train Loss: 0.6368, Train Accuracy: 77.11%, Val Loss: 0.8902, Val Accuracy: 69.98%
models saved to ViT/models/20241003_124314.pth


In [3]:
# Here is the hyperparameters
# 150-200
epochs = 50
patch_size = 8
embed_dim = 256
n_layers = 6
heads = 8
attn_dim = 512
mlp_dim = None # default to 4*embed_dim
pool = 'cls'
dropout = 0.0

model = ViT(image_size=32, patch_size=patch_size, num_classes=10, embed_dim=embed_dim, n_layers=n_layers, heads=heads, attn_dim=attn_dim, mlp_dim=mlp_dim, pool=pool, dropout=dropout)

last_time_str = "20241003_124314"

# load model
model.load_state_dict(torch.load(f"ViT/models/{last_time_str}.pth"))

print(f"models loaded from ViT/models/{last_time_str}.pth")

optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, betas=(0.9, 0.999), weight_decay=5e-4)

time_str = timestr()

print(f"Time string: {time_str}")

# print the model and the number of parameters
# print(model.transformer)
print(f'The model has {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters')

train(epochs=epochs, model=model, optimizer=optimizer, criterion=nn.CrossEntropyLoss(), 
      train_loader=train_loader, val_loader=val_loader, outdir=get_outdir(time_str))

torch.save(model.state_dict(), f"ViT/models/{time_str}.pth")

print(f"models saved to ViT/models/{time_str}.pth")

Epoch 1/50:   0%|                                                                               | 0/196 [00:00<?, ?it/s]

models loaded from ViT/models/20241003_124314.pth
Time string: 20241003_143919
The model has 6,356,234 trainable parameters


Epoch 2/50:   0%|                                                                               | 0/196 [00:00<?, ?it/s]

Epoch 1/50, Train Loss: 0.5556, Train Accuracy: 80.04%, Val Loss: 0.8447, Val Accuracy: 71.77%


Epoch 3/50:   1%|▎                                                    | 1/196 [00:00<00:37,  5.23it/s, Train Loss=0.543]

...

Epoch 20/50, Train Loss: 0.4559, Train Accuracy: 83.64%, Val Loss: 0.8814, Val Accuracy: 72.28%


Epoch 22/50:   0%|                                                                              | 0/196 [00:00<?, ?it/s]

Epoch 21/50, Train Loss: 0.4508, Train Accuracy: 83.75%, Val Loss: 0.8826, Val Accuracy: 71.66%


Epoch 22/50:   1%|▎                                                   | 1/196 [00:01<04:12,  1.30s/it, Train Loss=0.383]

: 